## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chapais,CA,49.7834,-74.8492,78.51,38,8,13.29,clear sky
1,1,Cidreira,BR,-30.1811,-50.2056,61.47,66,10,10.13,clear sky
2,2,Yellowknife,CA,62.4560,-114.3525,65.93,55,100,13.67,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,80.56,82,88,10.00,overcast clouds
4,4,East London,ZA,-33.0153,27.9116,54.54,94,0,3.44,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame
# using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chapais,CA,49.7834,-74.8492,78.51,38,8,13.29,clear sky
3,3,Puerto Ayora,EC,-0.7393,-90.3518,80.56,82,88,10.00,overcast clouds
11,11,Hobyo,SO,5.3505,48.5268,77.13,80,97,23.06,overcast clouds
12,12,Bengkulu,ID,-3.8004,102.2655,75.31,89,19,4.32,few clouds
14,14,Albany,US,42.6001,-73.9662,78.17,65,25,5.06,scattered clouds
15,15,Itabira,BR,-19.6192,-43.2269,82.27,34,100,9.71,overcast clouds
16,16,Delfzijl,NL,53.3300,6.9181,76.01,71,75,11.65,broken clouds
21,21,Kapaa,US,22.0752,-159.3190,80.11,80,75,14.97,broken clouds
22,22,Broome,US,42.2506,-75.8330,79.02,62,100,9.22,overcast clouds
23,23,Sembe,CG,1.6481,14.5806,75.60,79,73,3.20,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Step npot needed since the check in Step 4a showed no null values.

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, 
# max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", 
                                "Max Temp", "Current Description", 
                                "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chapais,CA,78.51,clear sky,49.7834,-74.8492,
3,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,
11,Hobyo,SO,77.13,overcast clouds,5.3505,48.5268,
12,Bengkulu,ID,75.31,few clouds,-3.8004,102.2655,
14,Albany,US,78.17,scattered clouds,42.6001,-73.9662,
15,Itabira,BR,82.27,overcast clouds,-19.6192,-43.2269,
16,Delfzijl,NL,76.01,broken clouds,53.3300,6.9181,
21,Kapaa,US,80.11,broken clouds,22.0752,-159.3190,
22,Broome,US,79.02,overcast clouds,42.2506,-75.8330,
23,Sembe,CG,75.60,broken clouds,1.6481,14.5806,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found 
    #     skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print('Hotel Not Found... Skipping')
print('Completed Hotel Search')

Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Hotel Not Found... Skipping
Completed Hotel Search


In [56]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chapais,CA,78.51,clear sky,49.7834,-74.8492,Hôtel Opémiska
3,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Hobyo,SO,77.13,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
12,Bengkulu,ID,75.31,few clouds,-3.8004,102.2655,Grage Hotel Bengkulu
15,Itabira,BR,82.27,overcast clouds,-19.6192,-43.2269,Premium Executive Hotel
16,Delfzijl,NL,76.01,broken clouds,53.3300,6.9181,Eemshotel
21,Kapaa,US,80.11,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
22,Broome,US,79.02,overcast clouds,42.2506,-75.8330,Chenango Valley State Park
24,Salinas,US,81.99,clear sky,36.6777,-121.6555,Laurel Inn & Conference Center
29,Kencong,ID,75.00,overcast clouds,-8.2833,113.3667,GUBUK SUGIANTO


In [59]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [60]:
# 9. Using the template add city name, the country code, the weather description and maximum 
#    temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data 
#      in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [65]:
# 11a. Add a marker layer for each city to the map. 

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the marker layer variable.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))